In [13]:
import numpy as np
import pandas as pd
from pandas_datareader import data as web
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
%matplotlib inline
import datetime as dt
import mplfinance as mpf
import yfinance as yf

In [ ]:
#pip install yfinance

# **Function to save stock data to CSV**


In [7]:
def save_to_csv_from_yahoo(ticker, sdate, edate):
  #start = dt.datetime(sdate)
  #end = dt.datetime(edate)

  #tickerData = yf.Ticker(ticker)
  
  # get historical market data
  #df = tickerData.history(period="max")
  yfdf = yf.download(ticker, start=sdate, end=edate)
  #yfdf = yf.download('gbtc', start="2021-01-01", end="2021-01-01")
  #print(yfdf)
  mydf = pd.DataFrame(yfdf)
  

  #yfdf.actions.to_csv('D:\\py\\pycode\\banas-tut01\\' + ticker + '.csv')
  #csvresult = mydf.to_csv('D:\\py\\pycode\\banas-tut01\\' + ticker + '.csv')
  #csvResult = df.to_csv("d:\\gbtc.csv")
  mydf.to_csv('d:/' + ticker + '.csv')
  #csvresult = mydf.to_csv('gbtc.csv')
  
  return mydf

In [ ]:
 #save_to_csv_from_yahoo("gbtc", "2021-01-01", "2021-11-03")

# **Get data from CSV**

In [11]:
def get_df_from_csv(ticker):
    try:
        df = pd.read_csv('d:/' + ticker + '.csv')
    except FileNotFoundError:
        print("File not found")
    else:
        return df


# **Add daily return to DF**

In [16]:
# simple daily return: (Ending Prid - Beginning Price)/Beginning Price or (EP/BP)-1

def add_daily_return_to_df(df,ticker):
    df['Daily_Return'] = (df['Adj Close'] / df['Adj Close'].shift(1)) - 1
    df.to_csv('d:/' + ticker + '.csv')
    return df


# **Returns Total Return over Time**

In [15]:
def get_return_defined_time(df, syear, smonth, sday, eyear, emonth, eday):
    start = f"{syear}-{smonth}-{sday}"
    end = f"{eyear}-{emonth}-{eday}"
    mask = (df['Date'] >= start) & (df['Date'] <= end)
    daily_ret = df.loc[mask]['Daily_Return'].mean()
    df2 = df.loc[mask]
    days = df2.shape[0]
    return (days * daily_ret)

# **Matplotlib Finance**

In [ ]:
def mplfinance_plot(ticker, chart_type, syear, smonth, sday, eyear, emonth, eday):
    start = f"{syear}-{smonth}-{sday}"
    end = f"{eyear}-{emonth}-{eday}"
    try:
        df = pd.read_csv('d:/' + ticker + '.csv')
    except FileNotFoundError:
        print("File not found")
    else:

# **Test Functions**

In [17]:
GBTC = get_df_from_csv("GBTC")

add_daily_return_to_df(GBTC,"GBTC")

#GBTC

tot_ret = get_return_defined_time(GBTC, 2020, 1, 1, 2021, 1, 1)
print("Total Return :", tot_ret)

Total Return : 0.2611085404296556
